# Sistema de Recomendación de Películas

Este proyecto implementa un sistema de recomendación de películas basado en contenido, usando el dataset TMDB 5000. Permite buscar películas por títulos parciales o con errores tipográficos (por ejemplo, "The lord of rings") y recomienda películas similares basadas en géneros, palabras clave, elenco, director y sinopsis. Utiliza TF-IDF para vectorización y similitud coseno para encontrar películas relacionadas.



## 1. Importación de Librerías

Se importan las librerías necesarias para el procesamiento de datos, vectorización, cálculo de similitud y búsqueda difusa de títulos.

In [26]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process

In [24]:
%pip install fuzzywuzzy
%pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.7 MB/s eta 0:00:00


## 2. Carga y Unión de Datasets

Cargamos los archivos `tmdb_5000_movies.csv` y `tmdb_5000_credits.csv`, y los unimos por el campo `id` para combinar información de películas (sinopsis, géneros) con créditos (elenco, director).

In [27]:
# Cargar datasets
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# Unir por 'id' (renombramos 'movie_id' en credits a 'id')
credits = credits.rename(columns={'movie_id': 'id'})
df = movies.merge(credits, on='id')

# Seleccionar columnas útiles
df = df[['id', 'title_x', 'overview', 'genres', 'keywords', 'cast', 'crew']]
df = df.rename(columns={'title_x': 'title'})


## 3. Limpieza de Datos

Limpiamos los datos manejando valores nulos y convirtiendo campos JSON (genres, keywords, cast, crew) en listas utilizables. Extraemos hasta 3 géneros, palabras clave y actores por película, y el nombre del director.


In [29]:
# Limpiar nulos
df['overview'] = df['overview'].fillna('')

# Función para extraer nombres de géneros y keywords
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        return names[:3] if len(names) > 3 else names
    return []

# Función para extraer director de crew
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Función para extraer actores de cast
def get_actors(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        return names[:3] if len(names) > 3 else names
    return []

# Aplicar conversiones (genres, keywords, cast y crew son strings JSON)
features = ['genres', 'keywords', 'cast', 'crew']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

df['genres'] = df['genres'].apply(get_list)
df['keywords'] = df['keywords'].apply(get_list)
df['cast'] = df['cast'].apply(get_actors)
df['director'] = df['crew'].apply(get_director)
df['director'] = df['director'].fillna('').apply(lambda x: [x])


## 4. Creación de la Sopa de Características

Combinamos géneros, palabras clave, elenco, director y sinopsis en una sola columna de texto (`soup`) para cada película. Esto se usará para vectorización.

In [30]:

# Combinar todas las características en una sola columna 'soup'
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['genres']) + ' ' + x['overview']

df['soup'] = df.apply(create_soup, axis=1)



## 5. Vectorización y Cálculo de Similitud
Convertimos la columna `soup` en vectores TF-IDF para representar las características de las películas. Luego, calculamos la similitud coseno entre todas las películas para medir cuán parecidas son.

In [ ]:
# Vectorizar la 'soup'
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['soup'])

# Calcular matriz de similitud coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)




## 6. Búsqueda Difusa y Recomendaciones

Creamos una función que busca el título más cercano al ingresado (usando fuzzy matching) y devuelve las 10 películas más similares según la similitud coseno. Si no se encuentra una coincidencia razonable, se muestra un mensaje de error.



In [31]:
# Crear un mapeo de títulos a índices
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# Función de recomendación con búsqueda difusa
def get_recommendations(title, cosine_sim=cosine_sim, threshold=80):
    # Buscar el título más cercano usando fuzzywuzzy
    match = process.extractOne(title, df['title'], score_cutoff=threshold)
    if not match:
        return "No se encontró ninguna película que coincida con el título proporcionado."

    matched_title, score, _ = match
    idx = indices[matched_title]

    # Obtener recomendaciones basadas en el título encontrado
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 (excluyendo la película misma)

    movie_indices = [i[0] for i in sim_scores]
    recommendations = df['title'].iloc[movie_indices].tolist()

    return f"Película encontrada: {matched_title}\nRecomendaciones:\n" + '\n'.join([f"{i+1}. {movie}" for i, movie in enumerate(recommendations)])


## 7. Ejemplo de Uso

Probamos el sistema con un título parcial o con errores tipográficos, como "The lord of rings", para obtener recomendaciones.

In [32]:
movieName = "Movie: "
# Ejemplo de uso
print(get_recommendations(input(movieName)))  # Prueba con una película conocida

Movie: Harry Potter
Película encontrada: Harry Potter and the Half-Blood Prince
Recomendaciones:
1. Harry Potter and the Goblet of Fire
2. Harry Potter and the Order of the Phoenix
3. Harry Potter and the Chamber of Secrets
4. Harry Potter and the Prisoner of Azkaban
5. Harry Potter and the Philosopher's Stone
6. Into the Woods
7. Driving Lessons
8. The Little Prince
9. Krull
10. Sinbad: Legend of the Seven Seas
